# Informe EDA

In [69]:
%run lib.ipynb

# Lectura del dataset de películas
df_movies = pd.read_parquet("datasets/movies_dataset.parquet")
df_movies.info()
df_movies.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45376 entries, 0 to 45375
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   belongs_to_collection  4488 non-null   object 
 1   budget                 45376 non-null  object 
 2   genres                 45376 non-null  object 
 3   id                     45376 non-null  object 
 4   original_language      45365 non-null  object 
 5   overview               44435 non-null  object 
 6   popularity             45376 non-null  float64
 7   production_companies   45376 non-null  object 
 8   production_countries   45376 non-null  object 
 9   release_date           45376 non-null  object 
 10  revenue                45376 non-null  float64
 11  runtime                45130 non-null  float64
 12  spoken_languages       45376 non-null  object 
 13  status                 45296 non-null  object 
 14  tagline                20398 non-null  object 
 15  ti

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,release_year,return,populartiy
0,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,...,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,None,Toy Story,7.7,5415.0,1995,12.451801,21.946943
1,None,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,en,When siblings Judy and Peter discover an encha...,17.015539,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,...,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,1995,4.043035,17.015539
2,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602,en,A family wedding reignites the ancient feud be...,11.712900,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,...,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0,1995,0.000000,11.712900
3,None,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,...,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0,1995,5.090760,3.859495
4,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",11862,en,Just when George Banks has recovered from his ...,8.387519,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,...,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173.0,1995,0.000000,8.387519


## Chequeo de nulos

In [70]:
#suma de valores nulos por columna
print(columnas_de_nulos(df_movies))


belongs_to_collection    40888
original_language           11
overview                   941
runtime                    246
status                      80
tagline                  24978
dtype: int64


### De un total de 45376, la columna "belongs_to_collection" tiene un 90.1% de valores nulos, como además la columna no será utilizada en el API (ni en el recomendador), será eliminada

In [71]:
df_movies = df_movies.drop(['belongs_to_collection'], axis=1)
print(columnas_de_nulos(df_movies))

original_language       11
overview               941
runtime                246
status                  80
tagline              24978
dtype: int64


### Para la columna "original_language", tenemos 11 filas con valor nulo. 8 de ellos tienen sólo un valor en la columna "spoken_languages" así que lo imputaremos en "original_languaje", para las 4 restantes imputaremos el primer valor que se encuentre en #spoken_languajes". Esta columna es de interes ya que posteriormente la utilizaré para reducir el dataset y poder factibilizar la ejecución del algoritmo de recomendación, ya que con todo el dataset genera problemas de memoria.

In [72]:
df_movies.loc[[21579, 22805, 32103], ['original_language']] = 'en'
df_movies.loc[[19559, 44494], ['original_language']] = ''
df_movies.loc[[37354], ['original_language']] = 'cs'
df_movies.loc[[40982], ['original_language']] = 'ur'
df_movies.loc[[41802, 44573], ['original_language']] = 'xx'
df_movies.loc[[43978], ['original_language']] = 'fr'
df_movies.loc[[44328], ['original_language']] = 'sv'

print(columnas_de_nulos(df_movies))

overview      941
runtime       246
status         80
tagline     24978
dtype: int64


### Utilizaré la columna "overview" como la base del algoritmo de recomendación, por lo que necesito que este poblada para todas las películas. Utilizaré en primera instancia la columna "tagline" y posteriormente la columna "title".

In [73]:

#Reemplazo los valores nulos de la columna overview por su correspondiente en tagline
df_movies["overview"] = df_movies["overview"].fillna(df_movies["tagline"])

#Reemplazo los valores nulos de la columna overview por su correspondiente en title
df_movies["overview"] = df_movies["overview"].fillna(df_movies["title"])

print(columnas_de_nulos(df_movies))

runtime      246
status        80
tagline    24978
dtype: int64


## Valores duplicados

## Valores faltantes

## Tipos de datos

## Análisis estadístico

## Análisis de correlaciones